# Introduction
In this part of the tutorial we will use the `dna-triangle`s that we made in Part 1 to make the DNA tile of [Tikhomirov et al](https://doi.org/10.1038/nnano.2016.256), shown below. To do so we will use 4 of the `dna-triangle` created in part 1 to implement a the four triangles that make up the tile. We create a `dna-tile` class to create the abstraction  that representing a DNA tile.

![DNA Tile](greg-tile.png)

During this we show case some of the geometric manipulation functionality that `small` provides and takes a deeper look at the parent-child hierarchy. Finally we will demonstrate how to create staples to join the triangle and tiles together and how to set the sequence of our `dna` objects. 

Lets get started! First we need to import the definitions of the `dna-triangle` that we created in Part 1. To do we we use the `load` function to load in the whole file. Note we load the .lisp file with the code from part, not the .ipynb file we were working on, although these contain the same code.


In [6]:
(load "./part1.lisp")
(in-package :small)

To load "small":
  Load 1 ASDF system:
    small


T

#<PACKAGE "SMALL">


; Loading "small"

[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::AI in DEFUN

#<FUNCTION AI>
  [compiled function]


Lambda-list: (I)
Derived type: (FUNCTION (T) *)
Documentation:
  Calculate the number of base pairs in the i-th of the a DNA triangle that makes up the DNA square
Source file: /home/dd/quicklisp/local-projects/small/examples/dna-array/./part1.lisp
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::TRI-AX-X in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::TRI-AX-Y in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::TRI-AX-Z in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::TRI-AX-COORDS in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::THETA-1IJ in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::THETA-1IJ-SCAFFOLD in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::TRI-SCAF-COORDS in DEFUN
[SB-KERNEL:REDEFINITION-WITH-DEFUN] redefining SMALL::TRI-SCAFFOLD-HELIX in DEFUN
[SB-KERNEL:REDEFI

# Creating The `dna-tile` Classs
First we create a class to hold the DNA Tile abstraction. This is similar to what we did in part 1 where we created the `dna-triangle` to represent the DNA triangle.


In [7]:
(defclass/std dna-tile (dna-origami)
  ((t1 :doc "triangle 1" :std (make-instance 'dna-triangle))
   (t2 :doc "triangle 2" :std (make-instance 'dna-triangle))
   (t3 :doc "triangle 3" :std (make-instance 'dna-triangle))
   (t4 :doc "triangle 4" :std (make-instance 'dna-triangle))
   (stap-bridges :doc "Staple Bridges"))
  (:documentation "An implementation the DNA tile of Tikhomirov et al https://www.nature.com/articles/nnano.2016.256"))

#<STANDARD-CLASS SMALL::DNA-TILE>

Notice that now we have added the custom slots `t1`-`t4` to our `dna-tile` class, whereas when defining the  `dna-triangle` we only had the slots we inherited from the `dna-origami` superclass. These slots are each initialized to hold an instance of the `dna-triangle` class, which we use to construct the tile the tile.

Next we rotate the `dna-triangle`s to position them to form the `dna-tile`. 

In [8]:
(defmethod initialize-instance :after ((obj dna-tile) &key)
  (with-accessors ((t1 t1) (t2 t2) (t3 t3) (t4 t4)) obj
    (let* ((rot90 (rotation-matrix (v3 0 1 0) (/ pi -2)))
	   (rot180 (rotation-matrix (v3 0 1 0) pi))
	   (rot270 (rotation-matrix (v3 0 1 0) (/ (* 3 pi) -2))))
      (rotate-obj t2 rot90)
      (rotate-obj t3 rot180)
      (rotate-obj t4 rot270))))

#<STANDARD-METHOD COMMON-LISP:INITIALIZE-INSTANCE :AFTER (DNA-TILE) {10027E7153}>

[SB-KERNEL:REDEFINITION-WITH-DEFMETHOD] redefining INITIALIZE-INSTANCE :AFTER (#<STANDARD-CLASS SMALL::DNA-TILE>) in DEFMETHOD



We used the [`with-accessors` function](http://www.lispworks.com/documentation/HyperSpec/Body/m_w_acce.htm) to retrieve the objects in the `t1`-`t4` slots of the `dna-tile`. `with-accessors` first argument is a list where each element of the list is a two-item list containing a variable name and the name of an accessor function for a class, the second is an instance of that class. The result is that the variable are bound to the slots of the object passed as the second argument (this is similar to the `let` function in that it binds variables). For more information on classes in Common Lisp [see here](http://www.gigamonkeys.com/book/object-reorientation-classes.html).

We then create rotation matrices to represent the rotation we want performed on the `dna-triangle`s. `small` provides the `rotation-matrix` function (defined in [linear-algebra.lisp file](https://github.com/DurhamSmith/small/blob/master/linear-algebra.lisp)) to easily create rotation matrices. It takes a vector, which is the axis to rotate around and an angle in radians and returns a 3x3 matrix representing a rotation of the angle around the 

Here we have used the `rotate-obj` function to rotate triangles 2-4. Applying geometric manipulations (translation, rotation) on a `chem-obj` (the fundamental superclass that `small` uses to represent chemical entities) that is `parent` to other objects effects all the `children`, however geometrically manipulating a child has no effect on the `parent` objects coordinates, only on that of the children. The function `all-tfms` retrieves the list transformations that will be applied to the object. We can see this parent-child behavior;


In [13]:
(with-accessors ((t1 t1) (t2 t2) (t3 t3) (t4 t4)) (make-instance 'dna-tile)
  (format t "~& T2: ~A ~%" (all-tfms t2))
  (format t "~& 5nt:T2 ~A ~%" (all-tfms (5nt t2)))
  (format t "~& Now we rotate the child  ~%")
  (rotate-obj (5nt t2) (rotation-matrix (v3 0 1 0) pi))
  (format t "~& T2: ~A ~%" (all-tfms t2))
  (format t "~& 5nt:T2 ~A ~%" (all-tfms (5nt t2))))

NIL


 T2: ((rotate
  . #<MATRIX/DOUBLE-FLOAT (3x3):
       0.000     0.000    -1.000
       0.000     1.000     0.000
       1.000     0.000     0.000>)) 

 5nt:T2 ((rotate
  . #<MATRIX/DOUBLE-FLOAT (3x3):
       0.000     0.000    -1.000
       0.000     1.000     0.000
       1.000     0.000     0.000>)) 

 Now we rotate the child  

 T2: ((rotate
  . #<MATRIX/DOUBLE-FLOAT (3x3):
       0.000     0.000    -1.000
       0.000     1.000     0.000
       1.000     0.000     0.000>)) 

 5nt:T2 ((rotate
  . #<MATRIX/DOUBLE-FLOAT (3x3):
       0.000     0.000    -1.000
       0.000     1.000     0.000
       1.000     0.000     0.000>)
 (rotate
  . #<MATRIX/DOUBLE-FLOAT (3x3):
      -1.000     0.000     0.000
       0.000     1.000     0.000
      -0.000     0.000    -1.000>)) 


We see that even after we rotate the `5nt` (the `dna-nt` at the 5' end of strand) the transformations applied to its `parent`, the `dna-triangle`, remain the same.

We can write the `dna-tiles` nucleotides out and see the tile that we have created in applicable viewer such as oxview.

In [15]:
(with-accessors ((t1 t1) (t2 t2) (t3 t3) (t4 t4)) (make-instance 'dna-tile)
  (wmdna "tile-v0" t1 t2 t3 t4))

NIL

![DNA Tile V0](tile-v0.png)